# 11章 事前学習済みモデルの利用
- ファインチューニング：学習済みモデルのパラメータを初期値として、全てのレイヤー関数で学習する手法
- 転移学習：学習済みモデルのうち、出力に近い部分のみ学習する方法

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
from tqdm import tqdm
import wandb
import yaml
from pythonlibs.torch_lib_wandb import *
print(README)

In [ ]:
# デバイスの割り当て
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# 分類先クラスの名称リスト
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# 分類先クラス数　今回は10になる
n_output = len(list(set(classes)))
# 結果確認
print(n_output)

In [ ]:
# Transformsの定義
# 学習データ用: 正規化に追加で反転とRandomErasingを実施
transform_train = transforms.Compose([
  transforms.Resize(112),
  transforms.RandomHorizontalFlip(p=0.5), 
  transforms.ToTensor(),
  transforms.Normalize(0.5, 0.5), 
  transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
])
# 検証データ用 : 正規化のみ実施
transform = transforms.Compose([
  transforms.Resize(112),
  transforms.ToTensor(),
  transforms.Normalize(0.5, 0.5)
])

In [ ]:
# データ取得用関数 Dataset
data_root = './data'
train_set = datasets.CIFAR10(
    root = data_root, train = True,
    download = False, transform = transform_train)
# 検証データの取得
test_set = datasets.CIFAR10(
    root = data_root, train = False, 
    download = False, transform = transform)

In [ ]:
# バッチサイズ指定
batch_size = 50
# データローダー
# 訓練用データローダー
# 訓練用なので、シャッフルをかける
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
# 検証用データローダー
# 検証時にシャッフルは不要
test_loader = DataLoader(test_set,  batch_size=batch_size, shuffle=False) 

### ResNet18の読み込み

In [ ]:
#  必要ライブラリのロード
from torchvision import models
# 事前学習済みモデルのロード
# pretraind = True で学習済みパラメータも一緒に読み込む
net = models.resnet18(pretrained = True)


In [ ]:
# ネットワークの概要表示
#print(net)

In [ ]:
# モデルのサマリー表示
net = net.to(device)
#summary(net,(100,3,112,112))

In [ ]:
print(net.fc)
print(net.fc.in_features)

### 最終レイヤー関数の付け替え

In [ ]:
# 乱数の初期化
torch_seed()
# 最終レイヤー関数の入力次元数を確認
fc_in_features = net.fc.in_features
# 最終レイヤー関数の付け替え
net.fc = nn.Linear(fc_in_features, n_output)
# 確認
#print(net)

In [ ]:
net = net.to(device)
#summary(net,(100,3,224,224))

In [ ]:
# 損失の計算グラフ可視化
criterion = nn.CrossEntropyLoss()
loss = eval_loss(test_loader, device, net, criterion)
g = make_dot(loss, params=dict(net.named_parameters()))
#display(g)

### 学習と結果評価

In [ ]:
# 乱数の初期化
torch_seed()

# 事前学習済みモデルのロード
# pretraind = True で学習済みパラメータも一緒に読み込む
net = models.resnet18(pretrained = True)

# 最終レイヤー関数の入力次元数を確認
fc_in_features = net.fc.in_features

# 最終レイヤー関数の付け替え
net.fc = nn.Linear(fc_in_features, n_output)

# GPUの利用
net = net.to(device)

# 学習率
lr = 0.001

# 損失関数定義
criterion = nn.CrossEntropyLoss()

# 最適化関数定義
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

# historyファイル初期化する
history = np.zeros((0, 5))

# 学習
num_epochs = 5

In [ ]:
def main():
    wandb.init()
    # 乱数の初期化
    torch_seed()
    # 事前学習済みモデルのロード
    # pretraind = True で学習済みパラメータも一緒に読み込む
    net = models.resnet18(pretrained = True)
    # 最終レイヤー関数の入力次元数を確認
    fc_in_features = net.fc.in_features
    # 最終レイヤー関数の付け替え
    net.fc = nn.Linear(fc_in_features, n_output)
    # GPUの利用
    net = net.to(device)
    # 学習率
    lr = 0.001
    # 損失関数定義
    criterion = nn.CrossEntropyLoss()
    # historyファイル初期化する
    history = np.zeros((0, 5))
    num_epochs = 20

    if wandb.config["optimizer"] == "SDG":
        # SGD
        optimizer = optim.SGD(net.parameters(), lr=lr)
        history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history, use_wandb=True)
    elif wandb.config["optimizer"] == "SDG_momentum":
        # SGD momentum
        optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
        history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history, use_wandb=True)
    elif wandb.config["optimizer"] == "Adagrad":
        # Adagrad
        optimizer = optim.Adagrad(net.parameters())
        history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history, use_wandb=True)
    elif wandb.config["optimizer"] == "Adam":
        # Adam
        optimizer = optim.Adam(net.parameters())
        history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history, use_wandb=True)
    elif wandb.config["optimizer"] == "RMSprop":
        # RMSprop
        optimizer = optim.RMSprop(net.parameters())
        history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history, use_wandb=True)
    elif wandb.config["optimizer"] == "Adadelta":
        # Adadelta
        optimizer = optim.Adadelta(net.parameters())
        history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history, use_wandb=True)
    elif wandb.config["optimizer"] == "AdamW":
        # AdamW
        optimizer = optim.AdamW(net.parameters())
        history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history, use_wandb=True)
    
    print(optimizer)

In [ ]:
#パラメータ読み込み
def yaml_read(yaml_file):
    with open(yaml_file) as f:
        cfg = yaml.safe_load(f)
    return cfg

In [ ]:
def sweep():
  sweep_config = yaml_read("config_sweep_trans.yaml")
  sweep_id = wandb.sweep(sweep_config, project="saitan_sweep_trans")
  wandb.agent(sweep_id, main)

In [ ]:
sweep()